# Neuroscape playground

In [1]:
from src.data import get_data
from src.utils import get_args_and_config
from src.train import train
from src.fmri import plot_brain

100%|██████████| 6/6 [00:02<00:00,  2.02it/s]


## data

In [2]:
args, config = get_args_and_config()
data = get_data(args, config)

100%|██████████| 6/6 [00:33<00:00,  5.56s/it]


In [3]:
params_lst = train(data, config)

wandb: Currently logged in as: syrkis. Use `wandb login --relogin` to force relogin


100%|██████████| 6000/6000 [01:52<00:00, 53.14it/s]


algonauts_train_lh_corr,▆▇▃█▆█▃▇▃▄▅▅▆▇▃▆▄▆▄▄▄▃▅▁▄▅▃▂▇▂▄▇▁▆▁▅▃▃▅▄
algonauts_train_rh_corr,▇▇▅█▆▇▂▆▅▃▃▆▅▇▃▇▅▆▅▆▄▃▆▁▄▇▄▂█▄▇█▃▇▄▆▄▄▅▄
algonauts_val_lh_corr,▃▁▂▂▃▆▃▆▄▇▇▅▆▅▄▆▅▅▆▆▆█▅▇▄▆▅█▄▇█▄█▅▅▇▇▇▃▄
algonauts_val_rh_corr,▄▁▄▃▄▇▄▃▄▆▆▄▅▆▄▅▇▆▅▃▅█▄▆▅▆▅▅▆▆▆▅▇▇▃█▆▆▄▂
train_cat_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_lh_corr,▁▂▃▅▅▅▄▅▅▆▆▅▆▇▆▆▇▇▇▇▇▇▇▇▇▆▆▆█▆▇█▆▇▇▇▇▆▆▆
train_lh_loss,█▃▃▃▂▂▃▂▂▂▂▂▂▁▂▂▂▂▂▁▂▂▁▂▂▂▁▂▂▁▁▂▂▁▂▁▁▂▁▂
train_loss,█▂▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rh_corr,▁▂▄▅▅▅▄▅▆▆▆▅▇▆▇▆▇▇▆▇▇▇▇▇▇▇▆▆█▇██▆▇█▇█▇▆▆
train_rh_loss,█▃▃▃▂▃▃▂▂▂▃▂▂▂▃▂▂▂▂▁▂▂▂▂▂▂▁▂▂▁▁▂▂▁▂▁▂▂▁▂
val_cat_loss,█▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▁▂▁▂▂▁▁▁


100%|██████████| 6000/6000 [01:43<00:00, 57.85it/s]


algonauts_train_lh_corr,▃█▅▃▆▄▃▆▆▃█▃▆▄▃▄▅▇▃▄▂▃▅▆▅▂█▆▃▄▆▄▆▄▁▃▁▃█▃
algonauts_train_rh_corr,▆▇▆▄▅▅▄▇▆▄▆▁▅▄▃▅▆▇▃▃▃▃▂▅▃▂█▅▃▄▆▃▆▄▁▃▃▃█▂
algonauts_val_lh_corr,█▃▅▂▃▆▅▅▅▄▅▁▆▅▃▇▆▄██▅▅▂▆▅▅▂▁▅▅▇▂▆▇▅▄▆▄▄▆
algonauts_val_rh_corr,▇▃▅▃▆▇▅▇▅▆▄▁▅▅▅▇▇▅▇▇▆▅▃▆▆▅▄▂▄▃▇▅▆█▆▇▇▅▆█
train_cat_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_lh_corr,▁▃▄▄▅▅▅▆▆▅▇▅▅▆▅▇▇▆▆▆▇█▇▇▆▇▇█▆▇▇██▇▆▇▇███
train_lh_loss,█▄▂▃▂▂▂▂▂▃▂▂▂▂▂▂▂▂▁▂▂▁▂▂▂▂▁▂▂▂▁▂▂▁▁▃▁▁▂▁
train_loss,█▂▁▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁
train_rh_corr,▁▃▅▃▄▅▆▆▆▅▆▅▅▅▅▇▇▆▆▆▆▇▆▇▆▆▇█▆▆▇█▇▆▆▇▇██▇
train_rh_loss,█▄▂▃▂▂▂▂▂▃▂▂▂▂▂▂▂▁▁▁▂▁▂▂▂▃▁▂▂▂▁▁▂▂▂▂▁▁▂▁
val_cat_loss,█▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▂▂▁▂▁▁▁▂▁▁▁▂▁▂▁▂▂▂▂▁▂▂▁▁


...

100%|██████████| 6000/6000 [01:44<00:00, 57.35it/s]


algonauts_train_lh_corr,▃▇▆▇▇▅▅▅▃█▇▅▄▃▄▆█▅▆▆▆▅▁▆▅▄▄▄▆▅▄▄▄▇▄▅▇▅▆▃
algonauts_train_rh_corr,▃▇▇▇▇▃▄▇▅█▇▆▄▃▅▅▆▄▆▇▅▅▁▆▅▅▅▅▅▇▄▆▄▆▄▄█▆▇▃
algonauts_val_lh_corr,▁▆▄▁▃▃▄▃▆▃▅▂▆▃▃▅▄▃▄▅▆▄▄▅▂▂█▃▅▇▆▄▁▁▃▅▃▅▆▆
algonauts_val_rh_corr,▁▆▅▁▃▄▅▃▆▃▄▃▅▄▃▇▅▃▆▆█▆▆▆▃▃█▄▄▆▇▆▂▂▅▆▆▄▆▅
train_cat_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_lh_corr,▁▃▃▅▅▄▄▅▄▇▆▅▆▇▆▆▅▇▆▇▇▆▆▆▆▆▇▆▇▇▆▇▇▇▆▇█▇▇▇
train_lh_loss,█▃▄▂▃▃▂▃▃▃▂▁▃▂▂▂▁▂▂▃▂▂▂▁▁▂▂▃▂▂▂▂▂▂▁▂▁▂▂▂
train_loss,█▂▂▂▂▂▂▂▂▂▁▁▂▁▁▂▁▂▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
train_rh_corr,▁▂▄▅▅▄▄▅▆▇▆▅▆▇▆▆▅▇▆▇▇▆▆▇▇█▇▇▇█▇█▇▇▆█████
train_rh_loss,█▃▅▃▃▃▃▃▃▃▂▂▃▂▂▂▂▃▃▃▂▃▂▁▂▂▂▃▂▂▂▂▂▂▂▂▁▂▂▃
val_cat_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▂▁▁▁▁▁▁▂▁▁▁▁▁▂▂▁▁▁▂▁▁


100%|██████████| 6000/6000 [02:06<00:00, 47.53it/s]


algonauts_train_lh_corr,▇█▄▆▁▃▆▆▅▅▇▅██▆▄▇▇█▅▅▆▆▅▄▅▇▆▅▅▆▆▅▆▇▇▆▆▇▄
algonauts_train_rh_corr,▇▇▃▆▁▁▄▆▄▅▆▄█▇▅▃▅▆█▅▄▅▅▄▅▅▇▅▅▆▅▅▅▄▇▆▅▄▅▄
algonauts_val_lh_corr,▅▅▃▃▄▅▃▅▅▅▃▁▂▄▄▅▃▃▅▅▂▃▅▁▆▇▃▄▆▄▂█▃▄▁▇▇▃▆▂
algonauts_val_rh_corr,▆█▅▆▄▆▃▅▇▄▃▃▃▇▅▆▃▆▆▆▃▂▆▃▇█▁█▇▆▁▆▁▆▂▇▇▄█▂
train_cat_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_lh_corr,▁▃▃▄▃▂▅▅▄▅▅▅▆▆▆▅▇▆▆▆▆▇▆▆▇██▆▇▆▆▇▆▆█▆▅▇█▇
train_lh_loss,█▃▃▃▄▄▃▃▄▂▂▂▂▂▂▃▃▃▂▂▃▃▃▂▂▁▂▂▄▂▂▂▁▂▂▃▂▃▂▂
train_loss,█▂▂▁▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rh_corr,▁▃▃▄▃▁▄▅▅▅▆▄▆▆▅▅▇▇▆▆▆▇▆▆▇██▆▇▆▆▇▆▇█▇▆▇█▇
train_rh_loss,█▃▃▃▄▄▃▃▃▂▂▂▂▂▃▃▂▂▂▂▃▃▃▂▂▁▂▂▄▂▂▂▁▂▁▃▁▂▂▂
val_cat_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▂▂▁▂▂▂▁▂▂▁▁▂▁▂▁


 94%|█████████▍| 5628/6000 [01:38<00:04, 79.47it/s]